# Example Usage of Scraipe Library

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import dependencies

In [ ]:
%pip install scraipe
#import sys; sys.path.append('scraipe')

ERROR: Could not find a version that satisfies the requirement scraipe (from versions: none)
ERROR: No matching distribution found for scraipe
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from scraipe import Workflow
from scraipe.scrapers import NewsScraper
from scraipe.analyzers import OpenAiAnalyzer
from pydantic import BaseModel

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [19]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
import time
print(f"Found {len(news_links_df)} news links on {time.strftime('%Y-%m-%d %H:%M:%S')}")
display(news_links_df.head())

Found 147 news links on 2025-03-24 21:54:25


,link
0,https://apnews.com/article/nationals-announcer...
1,https://apnews.com/article/pro-palestinian-wat...
2,https://apnews.com/article/firefly-aerospace-b...
3,https://apnews.com/article/climate-change-extr...
4,https://apnews.com/article/china-beijing-forbi...


## Configure Workflow
Now we'll configure the scraipe workflow using DefaultScraper and OpenAiAnalyzer.

Store your OpenAI key in a file named .openai_key before running this code block.

In [14]:
#===Configure NewsScraper===
# NewsScraper doesn't require any extra configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Load API key from a file
api_key = open(".openai_key").read().strip()

# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(api_key,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape content from news links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [15]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
scrape_store_df = workflow.get_scrapes()
display(scrape_store_df.iloc[0:1])

Scraping 142/142 new or failed links...


Scraping URLs: 100%|██████████| 142/142 [01:05<00:00,  2.16it/s]

Successfully scraped 142/142 links.


,link,content,success,error
0,https://apnews.com/article/nationals-announcer...,The Associated Press is an independent global ...,True,None


## Analyze content with OpenAI
Next we will analyze the stored scrapes.

In [16]:
# Analyze the scraped content
workflow.analyze()

# Dispaly the analyses
analysis_store_df = workflow.get_analyses()
display(analysis_store_df.head())

Analyzing 142/142 new or failed links...


Analyzing content: 100%|██████████| 142/142 [02:17<00:00,  1.03it/s]

Successfully analyzed 142/142 links.


,link,output,success,error
0,https://apnews.com/article/nationals-announcer...,{'celebrities': []},True,None
1,https://apnews.com/article/pro-palestinian-wat...,{'celebrities': []},True,None
2,https://apnews.com/article/climate-change-extr...,{'celebrities': []},True,None
3,https://apnews.com/article/china-beijing-forbi...,{'celebrities': ['Empress Dowager Cixi']},True,None
4,https://apnews.com/article/fact-check-misinfor...,"{'celebrities': ['Donald Trump', 'Joe Biden', ...",True,None


## Compile the results
Finally, let's export the completed analysis. 

In [17]:
export_df = workflow.export()
display(export_df)
export_df.to_csv('celebrities.csv', index=False)

,link,celebrities
0,https://apnews.com/article/nationals-announcer...,[]
1,https://apnews.com/article/pro-palestinian-wat...,[]
2,https://apnews.com/article/climate-change-extr...,[]
3,https://apnews.com/article/china-beijing-forbi...,[Empress Dowager Cixi]
4,https://apnews.com/article/fact-check-misinfor...,"[Donald Trump, Joe Biden, Elon Musk]"
...,...,...
137,https://apnews.com/article/liberty-or-death-pa...,"[Malcolm X, Donald Trump, Patrick Henry, John ..."
138,https://apnews.com/article/fish-doorbell-nethe...,[]
139,https://apnews.com/article/march-madness-trans...,"[Doug Stewart, Kevin Young, Marc VandeWetterin..."
140,https://apnews.com/article/colorado-woman-kill...,[]


## Analyze the results
Now you can conduct your own analysis on the structured data collected by the scraipe workflow.

In [18]:
# Load the extracted data
celebrities_df = pd.read_csv('celebrities.csv')
from ast import literal_eval
celebrities_df['celebrities'] = celebrities_df['celebrities'].apply(literal_eval)

# Explode the nested list of celebrities
celebrities_df = celebrities_df.explode('celebrities')
celebrities_df['celebrities'] = celebrities_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
celebrities_df = celebrities_df['celebrities'].value_counts().reset_index()
celebrities_df.columns = ['celebrity', 'mentions']
celebrities_df = celebrities_df.sort_values('mentions', ascending=False)
celebrities_df.head(10)

,celebrity,mentions
0,Donald Trump,57
1,Elon Musk,20
2,Joe Biden,6
3,Marco Rubio,5
4,Robert F. Kennedy Jr.,5
5,Barack Obama,5
6,Kamala Harris,4
7,Vladimir Putin,4
14,Mike Waltz,3
12,Volodymyr Zelenskyy,3
